In [27]:
from dataclasses import dataclass
from enum import Enum, auto
from typing import Self
from uuid import uuid4
from bamt.preprocessors import Preprocessor
import pandas as pd
from sklearn import preprocessing as pp

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import re


In [28]:
df = pd.read_csv("/Users/pishchulov/edu/matobes/НИР/outer_source/migration/ITMO-2/migforecasting/mig whereabouts/inflow LO.csv")

Выделим ОКТМО из Ленобласти

In [29]:
oktmos = map(str,df.oktmo.unique())

In [30]:
oktmos_lo = [x for x in oktmos if re.fullmatch(r"41\d+", x)]

In [31]:
del oktmos

In [32]:
len(oktmos_lo)

226

Определим численность населения. Проблема: удалось найти данные только за 2024 год

In [33]:
pop_size_df = pd.read_excel("/Users/pishchulov/edu/matobes/НИР/outer_source/численность МО/Сhisl_MO_01-01-2024_only_LO.xlsx")

In [34]:
pop_size_df

,Коды территорий,Unnamed: 1,население,городское,сельское
0,4100000000,Ленинградская область,2035762,1373533,662229
1,41754000 0 0,Сосновоборский городской округ,63462,63462,0
2,41754000001 1 0 0 0,г Сосновый Бор,63462,63462,0
3,41603000 0 0,Бокситогорский муниципальный район,50855,39107,11748
4,41603101 0 0,Городское поселение Бокситогорское,15764,15480,284
...,...,...,...,...,...
271,41648418 0 0,Сельское поселение Нурминское,3338,0,3338
272,41648430 0 0,Сельское поселение Лисинское,1875,0,1875
273,41648443 0 0,Сельское поселение Тельмановское,26781,0,26781
274,41648444 0 0,Сельское поселение Трубникоборское,1599,0,1599


In [35]:
def discard_to_8_chars(s):
    if len(s)!=10:
        return s
    if s[-2:]=="00":
        return s[:-2]        

In [36]:
pop_size_df["Коды территорий"] = pd.Series([discard_to_8_chars(x.replace(" ","")) for x in map(str, pop_size_df["Коды территорий"])], dtype=str)

In [37]:
pop_size_df_lo = pop_size_df[pop_size_df["Коды территорий"].isin(oktmos_lo)]


In [38]:
# set(oktmos_lo) - set(pop_size_df_lo["Коды территорий"])

Проблема: некоторые ОКТМО исчезли, потому что они эти населенные пункту переехали на новые коды. Проигнорируем.

In [39]:
del pop_size_df

In [40]:
pop_size_df_lo=pop_size_df_lo.drop(["городское", "сельское"], axis = 1)
pop_size_df_lo=pop_size_df_lo.set_axis(["oktmo", "name", "pop_size"], axis=1)
pop_size_df_lo

,oktmo,name,pop_size
1,41754000,Сосновоборский городской округ,63462
3,41603000,Бокситогорский муниципальный район,50855
4,41603101,Городское поселение Бокситогорское,15764
6,41603102,Городское поселение Пикалёвское,20161
8,41603155,Городское поселение Ефимовское,5417
...,...,...,...
271,41648418,Сельское поселение Нурминское,3338
272,41648430,Сельское поселение Лисинское,1875
273,41648443,Сельское поселение Тельмановское,26781
274,41648444,Сельское поселение Трубникоборское,1599
